In [1]:
import pandas as pd
import numpy  as np
import json


In [41]:
# Read in the 311 Service Requests dataset for 2017
all_complaint = pd.read_csv('complaint.csv', header=0, names=['created_date', 'borough', 'complaint_type', 'incident_zip'],
                        dtype = np.object)

# Read in the 2010 US Census Population By Zip Code dataset
population = pd.read_csv('ztca.csv', header=0,  names=['incident_zip','population'], 
                         dtype = np.object)


***

<font style="color:rgba(0, 0, 0, 0.8); font-size:18px; font-style: italic;">1. Consider only the 10 most common overall complaint types. For each borough, how many of each of those 10 types were there in 2017?</font>

<font style="color:rgba(0, 0, 0, 0.8); font-size:16px;"> Initial data exploration and preparation</font>

In [3]:
top_complaint = all_complaint

In [4]:
top_complaint.head()

,created_date,borough,complaint_type,incident_zip
0,2017-01-01T00:00:00.000,BROOKLYN,Rodent,11234
1,2017-01-01T00:00:00.000,BROOKLYN,Rodent,11249
2,2017-01-01T00:00:00.000,QUEENS,Rodent,11435
3,2017-01-01T00:00:00.000,QUEENS,Food Poisoning,11372
4,2017-01-01T00:00:00.000,QUEENS,Food Poisoning,11105


In [5]:
top_complaint.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2445422 entries, 0 to 2445421
Data columns (total 4 columns):
created_date      object
borough           object
complaint_type    object
incident_zip      object
dtypes: object(4)
memory usage: 74.6+ MB


In [6]:
top_complaint['borough'].unique()

array(['BROOKLYN', 'QUEENS', 'MANHATTAN', 'BRONX', 'STATEN ISLAND',
       'Unspecified'], dtype=object)

In [7]:
# Drop missing values 
top_complaint.isnull().sum()
top_complaint = top_complaint.dropna()

# Drop "Unspecified" zip code
top_complaint = top_complaint[top_complaint['borough'] != 'Unspecified']

In [8]:
top_complaint.shape

(2338802, 4)

In [9]:
# Top ten most frequently occurring complaint types
top_ten_complaints = top_complaint['complaint_type'].value_counts().index[:10]
top_ten_complaints

Index(['Noise - Residential', 'HEAT/HOT WATER', 'Illegal Parking',
       'Blocked Driveway', 'Street Condition', 'UNSANITARY CONDITION',
       'Noise - Street/Sidewalk', 'Water System', 'Noise', 'PAINT/PLASTER'],
      dtype='object')

In [10]:
# Consider only the 10 most common overall complaint types
top_complaint = top_complaint[top_complaint.complaint_type.isin(top_ten_complaints)]
top_complaint.shape

(1145513, 4)

In [11]:
# Confirm complaints DFcontains rows with only top complaints 
len(top_complaint.complaint_type.unique())

10

# Top 10 Complaints by Borough

In [12]:

top_complaint['complaint_type'].groupby(top_complaint['borough']).value_counts()

borough        complaint_type         
BRONX          HEAT/HOT WATER             68713
               Noise - Residential        57663
               Blocked Driveway           24574
               UNSANITARY CONDITION       24557
               PAINT/PLASTER              19493
               Illegal Parking            16122
               Noise - Street/Sidewalk    14025
               Street Condition           11155
               Water System                9965
               Noise                       3056
BROOKLYN       Noise - Residential        67629
               HEAT/HOT WATER             66977
               Illegal Parking            55380
               Blocked Driveway           49301
               UNSANITARY CONDITION       26654
               Street Condition           24875
               Noise - Street/Sidewalk    21313
               Water System               19508
               PAINT/PLASTER              19398
               Noise                      15227
M

***

<font style="color:rgba(0, 0, 0, 0.8); font-size:18px; font-weight: 600">2. Consider only the 10 most common overall complaint types.  For the 10 most populous zip codes, how many of each of those 10 types were there in 2017?</font>

In [13]:
# Consider only the 10 most common overall complaint types (the top_complaint DataFrame).
# Join the two DataFrames (top_complaint, population) based on the incident_zip key.

complaint_by_zipcode = top_complaint.merge(population)
complaint_by_zipcode['population'] = complaint_by_zipcode['population'].astype('int')
complaint_by_zipcode.head()

,created_date,borough,complaint_type,incident_zip,population
0,2017-01-01T00:00:59.000,BRONX,HEAT/HOT WATER,10467,97060
1,2017-01-01T00:30:40.000,BRONX,Noise - Residential,10467,97060
2,2017-01-01T00:53:21.000,BRONX,Illegal Parking,10467,97060
3,2017-01-01T01:05:17.000,BRONX,Blocked Driveway,10467,97060
4,2017-01-01T01:18:41.000,BRONX,Noise - Residential,10467,97060


In [14]:
complaint_by_zipcode.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1141111 entries, 0 to 1141110
Data columns (total 5 columns):
created_date      1141111 non-null object
borough           1141111 non-null object
complaint_type    1141111 non-null object
incident_zip      1141111 non-null object
population        1141111 non-null int64
dtypes: int64(1), object(4)
memory usage: 52.2+ MB


In [15]:
# Top 10 most populous zip codes 

top_populous_zipcode = complaint_by_zipcode.drop_duplicates(['incident_zip'])
top_populous_zipcode = top_populous_zipcode.sort_values(by='population', ascending=False)

top_ten_populous_zipcode = top_populous_zipcode['incident_zip'].values[:10]
top_ten_populous_zipcode

array(['11368', '11226', '11373', '11220', '11385', '10467', '10025',
       '11208', '11236', '11207'], dtype=object)

In [16]:
top_complaint_by_populous_zipcode = complaint_by_zipcode[
    complaint_by_zipcode.incident_zip.isin(top_ten_populous_zipcode)]
top_complaint_by_populous_zipcode.head()

,created_date,borough,complaint_type,incident_zip,population
0,2017-01-01T00:00:59.000,BRONX,HEAT/HOT WATER,10467,97060
1,2017-01-01T00:30:40.000,BRONX,Noise - Residential,10467,97060
2,2017-01-01T00:53:21.000,BRONX,Illegal Parking,10467,97060
3,2017-01-01T01:05:17.000,BRONX,Blocked Driveway,10467,97060
4,2017-01-01T01:18:41.000,BRONX,Noise - Residential,10467,97060


#  Top 10 complaints by 10 most populous zip codes 

In [35]:
# todo: sort by zip code 
top_complaint_by_populous_zipcode['complaint_type'].groupby(
    top_complaint_by_populous_zipcode['incident_zip']).value_counts()

incident_zip  complaint_type         
10025         HEAT/HOT WATER             2397
              Noise - Residential        2085
              Noise                      1398
              Noise - Street/Sidewalk    1224
              Illegal Parking             735
              UNSANITARY CONDITION        714
              Street Condition            628
              PAINT/PLASTER               604
              Water System                481
              Blocked Driveway            125
10467         HEAT/HOT WATER             6041
              Noise - Residential        5803
              UNSANITARY CONDITION       2192
              Blocked Driveway           2067
              PAINT/PLASTER              1955
              Illegal Parking             982
              Noise - Street/Sidewalk     712
              Street Condition            612
              Water System                560
              Noise                       285
11207         Noise - Residential        3

***

<font style="color:rgba(0, 0, 0, 0.8); font-size:18px; font-weight: 600">3. Considering all complaint types. Which boroughs are the biggest "complainers" relative to the size of the population in 2017? Meaning, calculate a complaint-index that adjusts for population of the borough.</font>

In [58]:

all_complaint.isnull().sum()
all_complaint = all_complaint.dropna()

# Drop "Unspecified" zip code
all_complaint = all_complaint[all_complaint['borough'] != 'Unspecified']

all_complaint_population = all_complaint.merge(population)
all_complaint_population['population'] = all_complaint_population['population'].astype('int')

In [63]:
all_complaint_population.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2328967 entries, 0 to 2328966
Data columns (total 5 columns):
created_date      object
borough           object
complaint_type    object
incident_zip      object
population        int64
dtypes: int64(1), object(4)
memory usage: 106.6+ MB


In [66]:
# Number of complaints per borough in 2017

complaint_borough = all_complaint_population.groupby(['borough']).agg({'complaint_type': 'count'})['complaint_type']
complaint_borough

borough
BRONX            432981
BROOKLYN         737358
MANHATTAN        461523
QUEENS           571607
STATEN ISLAND    125498
Name: complaint_type, dtype: int64

In [72]:
# The population of the boroughs of New York City 
population_borough = all_complaint_population.drop_duplicates(subset='incident_zip')
total_population_borough = population_borough.groupby(['borough']).agg({'population': 'sum'})['population']
total_population_borough

borough
BRONX            1437243
BROOKLYN         2504700
MANHATTAN        1595338
QUEENS           2390901
STATEN ISLAND     468730
Name: population, dtype: int64

# Which boroughs are the biggest "complainers"?

In [74]:
complaint_borough/total_population_borough

borough
BRONX            0.301258
BROOKLYN         0.294390
MANHATTAN        0.289295
QUEENS           0.239076
STATEN ISLAND    0.267740
dtype: float64